# Imports

In [1]:
from __future__ import annotations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

import mlr_utils
import ForwardStepCandidates_AKL as fsc_AKL
import loo_q2 as loo

# Using Copilot to understand code

In [ ]:
## separate files for exp data and comp data

comp_file = "Phosphine_library_DFT_props_191120_copy" 
comp_sheet = "selprops_use_2_bowls" 
num_par = 182 
par_start_col = 1   # 0-indexed
comp_num_samples = 1359 
y_label_col_comp = 0  # 0-indexed

exp_file = "exp_bowls" 
exp_sheet = "Sheet1"
exp_num_samples = 10 
response_col = 9  # 0-indexed
y_label_col_exp = 0  # 0-indexed

compinp = pd.read_excel(comp_file+".xlsx",comp_sheet,header=0,index_col=y_label_col_comp,nrows=comp_num_samples+1,usecols=list(range(0,(num_par+par_start_col))))
compinp.index = compinp.index.map(str)
expinp = pd.read_excel(exp_file+".xlsx",exp_sheet,header=4,index_col=y_label_col_exp,nrows=exp_num_samples,usecols=list(range(0,response_col+1)))
expinp.index = [i.zfill(4) for i in expinp.index.map(str)]

xlabelrow = True
verbose = True

X_names = list(compinp.iloc[0,par_start_col-1:num_par+par_start_col-1])
X_labels = list(compinp.columns)[par_start_col-1:num_par+par_start_col-1]
compinp.drop(index=compinp.index[0],inplace=True)
X_all = np.asarray(compinp[X_labels],dtype=np.float)
y_labels_comp = np.asarray(list(compinp.index),dtype=str)
compnan = np.isnan(X_all).any(axis=1)
y_labels_comp,X_all = y_labels_comp[~compnan],X_all[~compnan]

X_labelname = [" ".join(i) for i in zip(X_labels,X_names)] 
X_labelname_dict = dict(zip(X_labels,X_names))

resp_label = list(expinp.columns)[response_col-1]
y = np.asarray(expinp.iloc[:,response_col-1],dtype=np.float)
y_labels_exp = np.asarray(list(expinp.index),dtype=str)

mask_y = y.nonzero()[0]
mask_y = ~np.isnan(y)
mask_X = np.array([True if i in y_labels_comp else False for i in y_labels_exp])
mask = mask_y&mask_X
print("n_samples before removing empty cells: {}".format(len(y)))
print("Removing {} samples.".format(len(y)-sum(mask)))
y = y[np.array(mask)]
y_labels = y_labels_exp[np.array(mask)]

X = np.asarray(compinp.loc[y_labels],dtype=np.float)

if verbose:
    print("Shape X (all): {}".format(X_all.shape))
    print("Shape X (exp): {}".format(X.shape))
    print("Shape y (exp): {}".format(y.shape)) 
    print("Shape labels (exp): {}".format(y_labels.shape)) 
    print("First X (exp) cell: {}".format(X[0,0]))
    print("Last X (exp) cell:  {}".format(X[-1,-1]))
    print("First y: {}".format(y[0]))
    print("Last y:  {}".format(y[-1]))
    print("Last label exp: {}".format(y_labels[-1]))
    print("Last label comp: {}".format(y_labels_comp[-3:]))
    #print(inp.head())

# Actually read in data

In [ ]:
# Parameter file variables
parameters_file = "example_dataset" 
parameters_sheet = "Sheet1" 
num_par = 20
parameters_start_col = 2   # 0-indexed
parameters_num_samples = 100 
parameters_y_label_col = 0  # 0-indexed
parameters_header_rows = 0

# Response file variables
response_file = "example_dataset"
response_sheet = "Sheet1"
response_num_samples = 100 
response_col = 1 # 0-indexed
response_y_label_col = 0  # 0-indexed
response_header_rows = 1

# Actually start reading stuff into dataframes
parameters_df = pd.read_excel("./InputData/" + parameters_file + ".xlsx",
                              parameters_sheet,
                              header = parameters_header_rows,
                              index_col = parameters_y_label_col,
                              nrows = parameters_num_samples + 1,
                              #usecols = list(range(0, (num_par + parameters_start_col)))
                              )
response_df = pd.read_excel("./InputData/" + response_file + ".xlsx",
                            response_sheet,
                            header = response_header_rows,
                            index_col = response_y_label_col,
                            nrows = response_num_samples,
                            usecols = list(range(0, response_col + 1))
                            )

# Drop any columns before parameters_start_col that are not the index column
parameters_columns_to_keep = [col for col in range(0, len(parameters_df.columns)) if col >= parameters_start_col-1]
parameters_df = parameters_df.iloc[:,parameters_columns_to_keep]

# Combine the two dataframes into the master dataframe
response_df.drop(response_df.columns[0:response_col-1], axis = 'columns', inplace = True)
data_df = response_df.merge(parameters_df, left_index = True, right_index = True, how='inner')
data_df.columns.values[0] = 'response' # Converts the output column name from whatever it is on the spreadsheet
data_df.dropna(inplace = True) # This trims the dataframe down to only the rows relevant to this dataset

# Creates a dictionary to convert x# labels to full parameter names
x_names = list(parameters_df.iloc[0, :num_par])
x_labels = list(parameters_df.columns)[:num_par]
x_labelname_dict = dict(zip(x_labels, x_names))

print("Parameter file shape: {}".format(parameters_df.shape))
print("Final parameter quantity: {}".format(len(x_labels)))
print("Final experiment quantity: {}".format(data_df.shape[0]))
print("First parameter cell: {}".format(data_df[x_labels[0]].iloc[0]))
print("Last parameter cell:  {}".format(data_df[x_labels[-1]].iloc[-1]))
print("First y: {}".format(data_df.iloc[0,0]))
print("Last y:  {}".format(data_df.iloc[-1,0]))
print("First reaction label: {}".format(data_df.index[0]))
print("Last reaction label:  {}".format(data_df.index[-1]))

display(data_df)


# Start modeling

In [ ]:
split = "y_equidist"
test_ratio = 0.3 

# New train_test_splits set up returns the data_df index labels for the relevant points
training_set, test_set = mlr_utils.train_test_splits(data_df, split, test_ratio)

# Scale the parameters then puts them into scaled_data_df
scaler = StandardScaler()
scaler.fit(data_df.loc[training_set, :])
scaled_data = scaler.transform(data_df)
scaled_data_df = pd.DataFrame(scaled_data, index = data_df.index, columns = data_df.columns)
scaled_data_df[['response']] = data_df[['response']]

In [4]:
def plot_fit(y_train,
             y_pred_train,
             y_test,
             y_pred_test,
             y_extra_measured=[],
             y_extra_predicted=[],
             leg=True,
             sav=False,
             label="y",
             loo_pred=[],
             plot_lims=[0,0]):
    
    y_orig_min = np.min(np.hstack((y_train,y_test)))
    y_pred_min = np.min(np.hstack((y_pred_train,y_pred_test)))
    y_orig_max = np.max(np.hstack((y_train,y_test)))
    y_pred_max = np.max(np.hstack((y_pred_train,y_pred_test)))
    delta_x = 0.04 * (y_orig_max-y_orig_min)
    delta_y = 0.04 * (y_pred_max-y_pred_min)
           
    yy_fit = np.polyfit(y_train,y_pred_train,deg=1)
    yy_fit_line = yy_fit[1]+yy_fit[0]*y_train
    
    plt.figure(figsize=(5,5))
    plt.xlim([y_orig_min-delta_x,y_orig_max+delta_x])
    plt.ylim([y_pred_min-delta_y,y_pred_max+delta_y])

    if len(loo_pred) != 0:
        plt.scatter(y_train,loo_train,label="LOO",color="black",marker=".",facecolor='none',s=200)
    plt.scatter(y_train,y_pred_train,label="training",color="black",marker=".",s=200)
    plt.scatter(y_test,y_pred_test,label="test",color='red',marker=".",linewidth=3, s=200)
    if len(y_extra_measured)>0:
        plt.scatter(y_extra_measured,y_extra_predicted,label="validation",color='green',marker=".",linewidth=3, s=200)

    plt.plot(y_train,yy_fit_line,color="darkgrey",linestyle='--',dashes=[5,15])
    if leg:
        plt.legend(loc='lower right', fontsize=10)
    plt.xlabel(label+" measured",fontsize=18, fontweight='bold')
    plt.ylabel(label+" predicted",fontsize=18, fontweight='bold')
    
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
    num_ticks = 5
    if(plot_lims == [0,0]):
        ax_start = 0
        ax_end = max(max(y_train), max(y_test)) * 1.1
    else:
        ax_start = plot_lims[0]
        ax_end = plot_lims[1]
        
    ticks = np.linspace(ax_start, ax_end, num_ticks)
    ticks = np.round(ticks, decimals=1)

    plt.gca().set_xticks(ticks)
    plt.gca().set_yticks(ticks)
    plt.gca().set_aspect('equal')
    
    plt.gca().spines['right'].set_color('none')
    plt.gca().spines['top'].set_color('none')

    plt.gcf().tight_layout()
    
    if not sav:
        plt.show()  
    else:
        plt.savefig(sav, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Forward stepwise selection keeping a set of candidates at each step

n_steps = 3
n_candidates = 30
collin_criteria = 0.5 # this is R2
# skipfeatures = [] #["x4","x3"]

%notebook inline
plt.ioff()

modeling_data_df = scaled_data_df.drop(axis = 'columns', labels = 'y_class')
modeling_data_df = modeling_data_df.loc[training_set, :]

modeling_data_df = scaled_data_df.loc[training_set, :]

# I think this is the function that actually does the MLR
results,models,scores,sortedmodels,candidates = fsc_AKL.ForwardStep_py(modeling_data_df,'response',
                    n_steps=n_steps,n_candidates=n_candidates,collin_criteria=collin_criteria)

# Identify the best model from the ForwardStep algorithm
selected_model_terms = results.loc[0, "Model"] # Store a tuple of 'xIDs' for the best model
selected_model = models[selected_model_terms].model # Store the LinearRegression object for that model

# Break up the train/test data into smaller dataframes for easy reference
x_train = scaled_data_df.loc[training_set, selected_model_terms] # Dataframe containing just the parameters used in this model for the ligands used in the training set
x_test = scaled_data_df.loc[test_set, selected_model_terms] # Dataframe containing just the parameters used in this model for the ligands used in the test set
y_train = scaled_data_df.loc[training_set, 'response']
y_test = scaled_data_df.loc[test_set, 'response']

# Predict the train and test sets with the model
y_predictions_train = selected_model.predict(x_train)
y_predictions_test =  selected_model.predict(x_test)

# Calculate q2 and k-fold for the model
q2, loo_train = loo.q2_df(x_train,y_train)
kfoldscores = mlr_utils.repeated_k_fold(np.asarray(x_train), np.asarray(y_train), k=5, n=100)

# Print a bunch of stats about the model
print(f"\nSplit method: {split}")
print(f"Test ratio: {test_ratio}")

print(f'\nParameters:\n{selected_model.intercept_:10.4f} +')
for i, parameter in enumerate(selected_model_terms):
    print(f'{selected_model.coef_[i]:10.4f} * {parameter} {x_labelname_dict[parameter]}')

print(f"\nTraining R2  = {selected_model.score(x_train, y_train):.3f}")
print(f'Training Q2  = {q2:.3f}')
print(f"Training MAE = {metrics.mean_absolute_error(y_train,y_predictions_train):.3f}")
print("Training K-fold R2 = {:.3f} (+/- {:.3f})".format(kfoldscores.mean(), kfoldscores.std() ** 2))

print(f"\nTest R2      = {mlr_utils.r2_val(y_test,y_predictions_test,y_train):.3f}")
print(f'Test MAE     = {metrics.mean_absolute_error(y_test,y_predictions_test):.3f}')
    
# Plot the model
plot_fit(y_train, y_predictions_train, y_test, y_predictions_test, leg=False, sav=False, label="Yield (%)", loo_pred=loo_train)

# Copilot-assisted implementation of Ridge Regression